# "Smooth" statistics & smooth line plot

Statistics "smooth" are calculated on the sample of two continuous variables (i.e., sample of points or lines).
It interpolates data points to create a smoother curve.

This notebook uses definitions from [DataFrame](https://kotlin.github.io/dataframe/overview.html).

## Usage

The "Smooth" statistic proves beneficial in scenarios with over-plotting or noise,
simplifying the process of identifying inherent trends and patterns.
It can also be used to make a more pretty line with a small number of points.

## Arguments

* Input (mandatory):
    - `x` — numeric sample of input points `x` coordinates
    - `y` — numeric sample of input points `y` coordinates
* Parameters (optional):
    - `method: SmoothMethod` — smoothing model:
        - `SmoothMethod.Linear(confidenceLevel: Double)` — linear model
        - `SmoothMethod.Polynomial(degree: Int, confidenceLevel: Double)` — polynomial model
        - `SmoothMethod.LOESS(span: Double, loessCriticalSize: Int, samplingSeed: Long, confidenceLevel: Double)` —
          Local Polynomial Regression model
    - `smootherPointCount: Int` — number of sampled points

### Generalized signature

The specific signature depends on the function,
but all functions related to "smooth" statistic (which will be discussed further below —
different variations of `statSmooth()`, `smoothLine()`) have approximately the same signature with the arguments above:

```
statSmoothArgs := 
   x, 
   y,
   method: SmoothMethod = SmoothMethod.LOESS(),
   smootherPointCount: Int = 100
```

The possible types of `x` and `y` depend on where a certain function is used.
They can be simply `Iterable` (`List`, `Set`, etc.) or a reference to a column in a `DataFrame`
(`String`, `ColumnAccessor`) or the `DataColumn` itself.

## Output statistics

| name      | type   | description                                          |
|-----------|--------|------------------------------------------------------|
| Stat.x    | Double | `x` coordinate                                       |
| Stat.y    | Double | `y` coordinate                                       |
| Stat.yMin | Double | Lower point-wise confidence interval around the mean |
| Stat.yMax | Double | Upper point-wise confidence interval around the mean |
| Stat.se   | Double | Standard error                                       |

## StatSmooth plots

In [1]:
%useLatestDescriptors
%use dataframe
%use kandy

import org.apache.commons.math3.distribution.NormalDistribution
import org.apache.commons.math3.distribution.UniformRealDistribution
import kotlin.random.Random

In [2]:
// To generate the data, we use a standard java math library
// https://commons.apache.org/proper/commons-math/

// Generate line with formula
val xs = (-100..100).map { it / 50.0 }
val lineFormula = { x: Double -> 2.0 / (x * x + 0.5) }
// Generate noises from normal distribution
val noises = NormalDistribution(0.0, 0.1).sample(xs.size).toList()
val ys = xs.zip(noises).map { lineFormula(it.first) + it.second }
// And drop 2/3 points
val random = Random(42)
val (newXs, newYs) = xs.zip(ys).shuffled(random).take(xs.size * 1 / 3).sortedBy { it.first }.unzip()

In [3]:
// Gather them into the DataFrame
val df = dataFrameOf(
    "speed" to newXs,
    "efficiency" to newYs
)
df.head(5)

speed,efficiency
"-1,980000","0,501928"
"-1,920000","0,431872"
"-1,900000","0,382416"
"-1,880000","0,571812"
"-1,780000","0,534422"


`df` has a signature

| speed | efficiency |
|-------|------------|

Let's take a look at `StatSmooth` output DataFrame:

In [4]:
df.statSmooth("speed", "efficiency").head(5)

Stat,,,,
x,y,yMin,yMax,se
"-1,980000","0,253745","0,040217","0,467274","0,106917"
"-1,940000","0,298804","0,088492","0,509116","0,105307"
"-1,900000","0,344317","0,137204","0,551430","0,103705"
"-1,860000","0,390396","0,186465","0,594327","0,102112"
"-1,820000","0,437040","0,236271","0,637808","0,100528"


It has the following signature:

<table>
  <thead>
    <tr>
      <th alignt="left" colspan="5">Stat</th>
    </tr>
  </thead>
  <thead>
    <tr>
      <th>x</th>
      <th>y</th>
      <th>yMin</th>
      <th>yMax</th>
      <th>se</th>
    </tr>
  </thead>
</table>

As you can see, we got a `DataFrame` with one `ColumnGroup` called `Stat` which contains several columns with statics.
For `statSmooth`, each row corresponds to one of the line points.
`Stat.x` is the column with this point `x` coordinate.
`Stat.y` is points `y` coordinate; `Stat.yMin` — lower point of confidence level.
`Stat.yMax` — upper point of confidence level.
`Stat.se` — standard error.

`DataFrame` with "smooth" statistics is called `StatSmoothFrame`

### `statSmooth` transform

`statSmooth(statSmoothArgs) { /*new plotting context*/ }` modifies a plotting context —
instead of original data (no matter was it empty or not) new `StatSmooth` dataset
(calculated on given arguments. Inputs can be provided as `Iterable` or as dataset column reference —
by name as a `String`, as a `ColumnReference` or as a `DataColumn`) is used inside a new context
(original dataset and primary context are not affected —
you can add layers using initial dataset outside the `statSmooth` context).
Since the old dataset is irrelevant, we cannot use references for its columns.
But we can refer to the new ones.
They are all contained in the `Stat` group and can be called inside the new context:

In [5]:
plot {
    statSmooth(newXs, newYs) {
        // new `StatSmooth` dataset here
        area {
            // use `Stat.*` columns for mappings
            x(Stat.x)
            y(Stat.y)
        }
    }
    points {
        x(newXs)
        y(newYs)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="XcgQc3"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[-1.98,-1.94,-1.9,-1.8599999999999999,-1.82,-1.78,-1.74,-1.7,-1.66,-1.62,-1.58,-1.54,-1.5,-1.46,-1.42,-1.38,-1.3399999999999999,-1.2999999999999998,-1.26,-1.22,-1.18,-1.1400000000000001,-1.1,-1.06,-1.02,-0.98,-0.94,-0.8999999999999999,-0.8599999999999999,-0.8200000000000001,-0.78,-0.74,-0.7,-0.6599999999999999,-0.6199999999999999,-0.5799999999999998,-0.54,-0.5,-0.45999999999999996,-0.41999999999999993,-0.3799999999999999,-0.33999999999999986,-0.30000000000000004,-0.26,-0.21999999999999997,-0.17999999999999994,-0.1399999999999999,-0.09999999999999987,-0.06000000000000005,-0.020000000000000018,0.020000000000000018,0.06000000000000005,0.10000000000000009,0.14000000000000012,0.18000000000000016,0.2200000000000002,0.26000000000000023,0.30000000000000027,0.33999999999999986,0.3799999999999999,0.41999999999999993,0.45999999999999996,0.5,0.54,0.5800000000000001,0.6200000000000001,0.6600000000000001,0.7000000000000002,0.7400000000000002,0.7800000000000002,0.8200000000000003,0.8599999999999999,0.8999999999999999,0.94,0.98,1.02,1.06,1.1,1.1400000000000001,1.1800000000000002,1.2200000000000002,1.2600000000000002,1.3000000000000003,1.3400000000000003,1.38,1.42,1.46,1.5,1.54,1.58,1.62,1.6600000000000001,1.7000000000000002,1.7400000000000002,1.7800000000000002,1.8200000000000003,1.8599999999999999,1.9,1.94,1.98],
"y":[0.2537454545382096,0.29880386019425953,0.3443171020518987,0.39039586771676654,0.43703954329623995,0.4842695909092516,0.5321059941332207,0.58057062903216,0.629679585328539,0.6794452179215105,0.7298774030345838,0.7809840688230839,0.8327731679524449,0.8852548734684393,0.9384351675493174,0.992313438913837,1.0468887848603987,1.1021534889139661,1.1580811015075394,1.2146564054212174,1.2718300148367045,1.3286921839445065,1.3836948373964413,1.4407156538480939,1.503960595671842,1.5768923485618713,1.6489586473034943,1.7389783552854983,1.8383445524465705,1.9282278463661835,2.0198571027870864,2.119027175428851,2.2288975819591323,2.3450890643165176,2.4613376705073273,2.571379448537882,2.668950446414501,2.7477867121435073,2.80162429373122,2.82419923918396,2.8137397444360754,2.817985556456566,2.855206833300614,2.915746818879277,2.9397888622485264,2.9600247893798586,2.9649125810166237,2.958815106325551,2.9556268673743467,2.9280140174496005,2.8486427098379017,2.9661171305017593,2.939751981757298,2.897747180880985,2.867985208110624,2.8266836150672123,2.7667492283054305,2.699283505368391,2.6361789747117483,2.6070383299341318,2.666289690868523,2.7020104051406193,2.615483484928397,2.5110359601084067,2.4168574480037908,2.32729798807688,2.236863465091441,2.143411818111227,2.047490191761796,1.9531114536241925,1.8639898142408085,1.7755116113274592,1.6887025789964192,1.6057674270717577,1.5288060889187396,1.457508639350156,1.3888729982310328,1.324666409719806,1.2670837825290606,1.2116004273775858,1.1572545522638549,1.1035953452084655,1.0503491424585463,0.9974375788419705,0.9448095768984048,0.892425858492105,0.8402909498247144,0.7884101997451979,0.7367941731298673,0.6854539786047944,0.6344006554851787,0.5836453388276142,0.5331991876240437,0.48307336086641034,0.4332784855259818,0.38382306049132586,0.33471505263033596,0.28596850511147576,0.23764484595687768,0.1895757778638698]
},
"sampling":"none",
"position":"identity",
"geom":"area"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"d

In [6]:
df.plot {
    statSmooth(speed, efficiency, method = SmoothMethod.Polynomial(2), smootherPointCount = 250) {
        ribbon {
            x(Stat.x)
            yMin(Stat.yMin)
            yMax(Stat.yMax)
        }
    }
    // Dataset is not changed here
    points {
        x(speed)
        y(efficiency)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="QHiRjj"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"efficiency":[0.5019283554890666,0.4318717828697747,0.3824164494083576,0.5718118913255327,0.5344221878900967,0.7039065823288215,0.7194640509898627,0.6496545352645979,0.668807835600986,0.8065993238705416,0.8257386174886828,0.8016999526058181,0.944382301272628,0.7916466016945914,1.1471550785923545,1.0069597384537003,1.3557611325973489,1.4159765584609671,1.391058527780478,1.411268307264855,1.7048537915021054,2.0737635888317025,2.8490525106604014,3.1382126907404926,3.2130993214853776,3.340067453042271,3.1381669007756527,3.6582229986489887,3.643943978551948,3.8986453833680326,3.7533902484518027,4.0330806446091145,4.110871862808168,3.9933835646544082,3.9515149086987083,3.9227294639991634,3.5524898191841023,3.301400266432915,3.299535793944115,2.851112642061264,2.5952503606320216,2.0821397502137162,2.3525654389313195,1.8763392984462213,1.7639275710161053,1.7505465404780922,1.7837426286577025,1.7196889860186766,1.478652550337799,1.3063711156683782,1.4183656309402524,1.1034278126923744,1.26499595979066,1.0113804690467496,1.1323751509817408,1.3702758486855853,0.8620347383254326,0.9612326300782081,0.9183437738409894,0.7883710550263271,0.7715769462478345,0.8554200652165197,0.6721598560707885,0.3800735143773569,0.38689241222586607,0.5399990292881343,0.44294488953286404],
"speed":[-1.98,-1.92,-1.9,-1.88,-1.78,-1.74,-1.7,-1.64,-1.52,-1.48,-1.34,-1.3,-1.24,-1.22,-1.18,-1.16,-1.0,-0.96,-0.94,-0.86,-0.84,-0.74,-0.42,-0.4,-0.36,-0.34,-0.32,-0.24,-0.2,-0.16,-0.14,0.02,0.04,0.06,0.08,0.14,0.2,0.34,0.38,0.46,0.5,0.64,0.66,0.72,0.74,0.76,0.8,0.82,0.96,1.04,1.06,1.08,1.1,1.14,1.16,1.2,1.24,1.26,1.38,1.42,1.46,1.48,1.58,1.74,1.88,1.9,1.98]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"yMin",
"ymax":"yMax"
},
"stat":"identity",
"data":{
"yMin":[-0.6448595778028926,-0.5904521944638826,-0.5364809324994897,-0.4829458625124176,-0.4298470566852394,-0.37718458881969646,-0.3249585343769725,-0.27316897051894,-0.22181597615041299,-0.1708996319624152,-0.12042002047646627,-0.07037722608993136,-0.020771335122411805,0.028397564136782072,0.0771293813800768,0.1254240242319936,0.17328139819839478,0.22070140661462434,0.26768395059252886,0.31422892896635485,0.36033623823752864,0.4060057725182987,0.45123742347427,0.4960310802658092,0.5403866294883469,0.5843039551115774,0.6277829384175733,0.6708234579378414,0.7134253893893241,0.7555886056093959,0.7973129764898657,0.8385983689100378,0.8794446466688699,0.919851670416272,0.9598192975836084,0.9993473823134662,1.0384357753887523,1.0770843241612131,1.115292872479448,1.1530612606165265,1.1903893251973092,1.2272768991255938,1.2637238115112182,1.299729887597251,1.3352949486874437,1.3704188120740919,1.4051012909665004,1.4393421944202547,1.473141327267491,1.5064984900484253,1.539413478944368,1.571886085712497,1.603916097622683,1.6355032973966614,1.6666474631498915,1.6973483683364397,1.7276057816972668,1.7574194672123156,1.7867891840568026,1.8157146865621816,1.8441957241822173,1.8722320414646871,1.899823378029208,1.926969468551745,1.9536700427563565,1.979924825414781,2.005733536354466,2.031095890475691,2.056011597778434,2.080480363399678,2.1045018876618364,2.1280758661330377,2.151201989699981,2.1738799446541233,2.196109412791935,2.2178900715299914,2.239221594035648,2.260103649374054,2.280535902672252,2.3005180153010767,2.320049645075572,2.339130446474589,2.3577600708802082,2.3759381668375887,2.3936643803357636,2.4109383551098853,2.4277597329653173,2.4441281541238946,2.4600432575926012,2.4

### `smoothLine` layer

`smoothLine` layer is a shortcut for fast plotting a smoothed line:

In [7]:
val smoothLineLayerPlot = plot {
    smoothLine(newXs, newYs)
    layout.title = "`smoothLine()` layer"
}
smoothLineLayerPlot

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="29m9Sx"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"`smoothLine()` layer"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[-1.98,-1.94,-1.9,-1.8599999999999999,-1.82,-1.78,-1.74,-1.7,-1.66,-1.62,-1.58,-1.54,-1.5,-1.46,-1.42,-1.38,-1.3399999999999999,-1.2999999999999998,-1.26,-1.22,-1.18,-1.1400000000000001,-1.1,-1.06,-1.02,-0.98,-0.94,-0.8999999999999999,-0.8599999999999999,-0.8200000000000001,-0.78,-0.74,-0.7,-0.6599999999999999,-0.6199999999999999,-0.5799999999999998,-0.54,-0.5,-0.45999999999999996,-0.41999999999999993,-0.3799999999999999,-0.33999999999999986,-0.30000000000000004,-0.26,-0.21999999999999997,-0.17999999999999994,-0.1399999999999999,-0.09999999999999987,-0.06000000000000005,-0.020000000000000018,0.020000000000000018,0.06000000000000005,0.10000000000000009,0.14000000000000012,0.18000000000000016,0.2200000000000002,0.26000000000000023,0.30000000000000027,0.33999999999999986,0.3799999999999999,0.41999999999999993,0.45999999999999996,0.5,0.54,0.5800000000000001,0.6200000000000001,0.6600000000000001,0.7000000000000002,0.7400000000000002,0.7800000000000002,0.8200000000000003,0.8599999999999999,0.8999999999999999,0.94,0.98,1.02,1.06,1.1,1.1400000000000001,1.1800000000000002,1.2200000000000002,1.2600000000000002,1.3000000000000003,1.3400000000000003,1.38,1.42,1.46,1.5,1.54,1.58,1.62,1.6600000000000001,1.7000000000000002,1.7400000000000002,1.7800000000000002,1.8200000000000003,1.8599999999999999,1.9,1.94,1.98],
"y":[0.2537454545382096,0.29880386019425953,0.3443171020518987,0.39039586771676654,0.43703954329623995,0.4842695909092516,0.5321059941332207,0.58057062903216,0.629679585328539,0.6794452179215105,0.7298774030345838,0.7809840688230839,0.8327731679524449,0.8852548734684393,0.9384351675493174,0.992313438913837,1.0468887848603987,1.1021534889139661,1.1580811015075394,1.2146564054212174,1.2718300148367045,1.3286921839445065,1.3836948373964413,1.4407156538480939,1.503960595671842,1.5768923485618713,1.6489586473034943,1.7389783552854983,1.8383445524465705,1.9282278463661835,2.0198571027870864,2.119027175428851,2.2288975819591323,2.3450890643165176,2.4613376705073273,2.571379448537882,2.668950446414501,2.7477867121435073,2.80162429373122,2.82419923918396,2.8137397444360754,2.817985556456566,2.855206833300614,2.915746818879277,2.9397888622485264,2.9600247893798586,2.9649125810166237,2.958815106325551,2.9556268673743467,2.9280140174496005,2.8486427098379017,2.9661171305017593,2.939751981757298,2.897747180880985,2.867985208110624,2.8266836150672123,2.7667492283054305,2.699283505368391,2.6361789747117483,2.6070383299341318,2.666289690868523,2.7020104051406193,2.615483484928397,2.5110359601084067,2.4168574480037908,2.32729798807688,2.236863465091441,2.143411818111227,2.047490191761796,1.9531114536241925,1.8639898142408085,1.7755116113274592,1.6887025789964192,1.6057674270717577,1.5288060889187396,1.457508639350156,1.3888729982310328,1.324666409719806,1.2670837825290606,1.2116004273775858,1.1572545522638549,1.1035953452084655,1.0503491424585463,0.9974375788419705,0.9448095768984048,0.892425858492105,0.8402909498247144,0.7884101997451979,0.7367941731298673,0.6854539786047944,0.6344006554851787,0.5836453388276142,0.5331991876240437,0.48307336086641034,0.4332784855259818,0.38382306049132586,0.33471505263033596,0.28596850511147576,0.23764484595687768,0.1895757778638698]
},
"sampling":"none",
"position":"identity",
"g

In [8]:
// Compare it with `statSmooth` + usual `line`
val statSmoothAndLinePlot = plot {
    statSmooth(newXs, newYs) {
        line {
            x(Stat.x)
            y(Stat.y)
        }
    }
    layout.title = "`statSmooth()` + non-statistical `line` layer"
}
plotGrid(listOf(smoothLineLayerPlot, statSmoothAndLinePlot))

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="YsEMpV"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"layout":{
"name":"grid",
"ncol":2,
"nrow":1,
"fit":true,
"align":false
},
"figures":[{
"ggtitle":{
"text":"`smoothLine()` layer"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[-1.98,-1.94,-1.9,-1.8599999999999999,-1.82,-1.78,-1.74,-1.7,-1.66,-1.62,-1.58,-1.54,-1.5,-1.46,-1.42,-1.38,-1.3399999999999999,-1.2999999999999998,-1.26,-1.22,-1.18,-1.1400000000000001,-1.1,-1.06,-1.02,-0.98,-0.94,-0.8999999999999999,-0.8599999999999999,-0.8200000000000001,-0.78,-0.74,-0.7,-0.6599999999999999,-0.6199999999999999,-0.5799999999999998,-0.54,-0.5,-0.45999999999999996,-0.41999999999999993,-0.3799999999999999,-0.33999999999999986,-0.30000000000000004,-0.26,-0.21999999999999997,-0.17999999999999994,-0.1399999999999999,-0.09999999999999987,-0.06000000000000005,-0.020000000000000018,0.020000000000000018,0.06000000000000005,0.10000000000000009,0.14000000000000012,0.18000000000000016,0.2200000000000002,0.26000000000000023,0.30000000000000027,0.33999999999999986,0.3799999999999999,0.41999999999999993,0.45999999999999996,0.5,0.54,0.5800000000000001,0.6200000000000001,0.6600000000000001,0.7000000000000002,0.7400000000000002,0.7800000000000002,0.8200000000000003,0.8599999999999999,0.8999999999999999,0.94,0.98,1.02,1.06,1.1,1.1400000000000001,1.1800000000000002,1.2200000000000002,1.2600000000000002,1.3000000000000003,1.3400000000000003,1.38,1.42,1.46,1.5,1.54,1.58,1.62,1.6600000000000001,1.7000000000000002,1.7400000000000002,1.7800000000000002,1.8200000000000003,1.8599999999999999,1.9,1.94,1.98],
"y":[0.2537454545382096,0.29880386019425953,0.3443171020518987,0.39039586771676654,0.43703954329623995,0.4842695909092516,0.5321059941332207,0.58057062903216,0.629679585328539,0.6794452179215105,0.7298774030345838,0.7809840688230839,0.8327731679524449,0.8852548734684393,0.9384351675493174,0.992313438913837,1.0468887848603987,1.1021534889139661,1.1580811015075394,1.2146564054212174,1.2718300148367045,1.3286921839445065,1.3836948373964413,1.4407156538480939,1.503960595671842,1.5768923485618713,1.6489586473034943,1.7389783552854983,1.8383445524465705,1.9282278463661835,2.0198571027870864,2.119027175428851,2.2288975819591323,2.3450890643165176,2.4613376705073273,2.571379448537882,2.668950446414501,2.7477867121435073,2.80162429373122,2.82419923918396,2.8137397444360754,2.817985556456566,2.855206833300614,2.915746818879277,2.9397888622485264,2.9600247893798586,2.9649125810166237,2.958815106325551,2.9556268673743467,2.9280140174496005,2.8486427098379017,2.9661171305017593,2.939751981757298,2.897747180880985,2.867985208110624,2.8266836150672123,2.7667492283054305,2.699283505368391,2.6361789747117483,2.6070383299341318,2.666289690868523,2.7020104051406193,2.615483484928397,2.5110359601084067,2.4168574480037908,2.32729798807688,2.236863465091441,2.143411818111227,2.047490191761796,1.9531114536241925,1.8639898142408085,1.7755116113274592,1.6887025789964192,1.6057674270717577,1.5288060889187396,1.457508639350156,1.3888729982310328,1.324666409719806,1.2670837825290606,1.2116004273775858,1.1572545522638549,1.1035953452084655,1.0503491424585463,0.9974375788419705,0.9448095768984048,0.892425858492105,0.8402909498247144,0.7884101997451979,0.7367941731298673,0.6854539786047944,0.6344006554851787,0.5836453388276142,0.5331991876240437,0.48307336086641034,0.4332784855259818,0.38382306049132586,0.33471505263033596,0.28596850511147576

`smoothLine` uses `statSmooth` and `line` and performs coordinate mappings under the hood.
And we can customize `smoothLine` layer: `smoothLine()` optionally opens a new context,
where we can configure the line (as in the usual context opened by `line { ... }`) —
even change coordinate mappings from default ones.
`StatSmooth` dataset of `smoothLine` is also can be accessed here.

In [9]:
df.plot {
    smoothLine(speed, efficiency, SmoothMethod.LOESS(span = 0.1), 120) {
        // change a column mapped on `y` to `Stat.scaled`
        y(Stat.yMax)
        color = Color.RED
        width = 4.0
    }
    points {
        x(speed)
        y(efficiency)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="bwxMer"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"efficiency":[0.5019283554890666,0.4318717828697747,0.3824164494083576,0.5718118913255327,0.5344221878900967,0.7039065823288215,0.7194640509898627,0.6496545352645979,0.668807835600986,0.8065993238705416,0.8257386174886828,0.8016999526058181,0.944382301272628,0.7916466016945914,1.1471550785923545,1.0069597384537003,1.3557611325973489,1.4159765584609671,1.391058527780478,1.411268307264855,1.7048537915021054,2.0737635888317025,2.8490525106604014,3.1382126907404926,3.2130993214853776,3.340067453042271,3.1381669007756527,3.6582229986489887,3.643943978551948,3.8986453833680326,3.7533902484518027,4.0330806446091145,4.110871862808168,3.9933835646544082,3.9515149086987083,3.9227294639991634,3.5524898191841023,3.301400266432915,3.299535793944115,2.851112642061264,2.5952503606320216,2.0821397502137162,2.3525654389313195,1.8763392984462213,1.7639275710161053,1.7505465404780922,1.7837426286577025,1.7196889860186766,1.478652550337799,1.3063711156683782,1.4183656309402524,1.1034278126923744,1.26499595979066,1.0113804690467496,1.1323751509817408,1.3702758486855853,0.8620347383254326,0.9612326300782081,0.9183437738409894,0.7883710550263271,0.7715769462478345,0.8554200652165197,0.6721598560707885,0.3800735143773569,0.38689241222586607,0.5399990292881343,0.44294488953286404],
"speed":[-1.98,-1.92,-1.9,-1.88,-1.78,-1.74,-1.7,-1.64,-1.52,-1.48,-1.34,-1.3,-1.24,-1.22,-1.18,-1.16,-1.0,-0.96,-0.94,-0.86,-0.84,-0.74,-0.42,-0.4,-0.36,-0.34,-0.32,-0.24,-0.2,-0.16,-0.14,0.02,0.04,0.06,0.08,0.14,0.2,0.34,0.38,0.46,0.5,0.64,0.66,0.72,0.74,0.76,0.8,0.82,0.96,1.04,1.06,1.08,1.1,1.14,1.16,1.2,1.24,1.26,1.38,1.42,1.46,1.48,1.58,1.74,1.88,1.9,1.98]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"speed",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"efficiency",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"yMax"
},
"stat":"identity",
"data":{
"yMax":[0.53225524717522,0.5310995707622932,0.532596404701682,0.5402025675527056,0.5697785019534873,0.6149721345029242,0.6595679831072724,0.691531234737506,0.7143938745620723,0.7242065660578704,0.7255424358337638,0.7328212090542155,0.747620109565241,0.7653256843843496,0.7813052992793361,0.7958125861927124,0.8175792745630882,0.8433924781033861,0.8663610151389397,0.8795936618760457,0.8823124876011684,0.9191233669686855,0.9766778355860773,0.9989502510890033,1.0554593849699119,1.1095697241070692,1.170595456455035,1.2367332437207288,1.3028435025239862,1.3637866673243912,1.414004152091554,1.4415689794303108,1.4863609467139318,1.583746695783869,1.7201112729929617,1.8593613601902192,1.977869549404603,2.0773674876914656,2.1600349863903356,2.22978773318529,2.2914666731774127,2.3499213984015683,2.4100015083576434,2.4765566072866885,2.5544363010836624,2.648490193847643,2.7635678840796123,2.9044976144373535,3.0612089626908956,3.2203545106879417,3.526080991255961,3.690623896954529,3.6990423103122945,3.6866676771849383,3.8033389937041893,3.8779959923633993,3.82094441726111,3.823099619473469,3.8729680614724535,3.955703803535281,4.056460842315929,4.1204119663587395,4.002087523805954,3.9654257396419523,3.91111155690826,3.7358516422186114,3.556214985136015,3.436784640689818,3.3660453084482262,3.329013919142826,3.310595041970265,3.2856632166461885,3.2150093510210094,3.069316600337801,2.831264671600943,2.588941552755183,2.41286681508391,2.2829345122744846,2.174563373075771,2.063739010641352,1.9729479258395302,1.8988595053598005,1.7948215534055627,1.7845

## Grouped `statSmoth`

`statSmooth` can be applied for grouped data —
statistics will be calculated on each group independently but with equal categories.
This application returns a new `GroupBy`
dataset with the same keys as the old one but with `StatSmooth` groups instead of old ones.

In [10]:
// Generate two lines
val fA = { x: Double -> 0.02 * x * x * x - 0.2 * x * x + 0.1 * x + 2.1 }
val fB = { x: Double -> -0.1 * x * x * x + 0.5 * x * x - 0.8 }
val xRange = (-500..500).map { it / 100.0 }
val noisesA = NormalDistribution(0.0, 0.05).sample(xRange.size).toList()
val noisesB = NormalDistribution(0.0, 0.2).sample(xRange.size).toList()
val valuesA = xRange.zip(noisesA).map { fA(it.first) + it.second }
val valuesB = xRange.zip(noisesB).map { fB(it.first) + it.second }

val (xsA, ysA) = xRange.zip(valuesA).shuffled(Random(17)).take(xRange.size * 1 / 3).sortedBy { it.first }
    .unzip()
val (xsB, ysB) = xRange.zip(valuesB).shuffled(Random(17)).take(xRange.size * 1 / 3).sortedBy { it.first }
    .unzip()

In [11]:
// Gather them into `DataFrame` with "A" and "B" keys in "category" column
val valuesDF = dataFrameOf(
    "time" to xsA + xsB,
    "value" to ysA + ysB,
    "category" to List(xsA.size) { "A" } + List(xsB.size) { "B" }
)
valuesDF.head(5)

time,value,category
"-4,960000","-5,737070",A
"-4,890000","-5,486676",A
"-4,870000","-5,426111",A
"-4,840000","-5,341519",A
"-4,830000","-5,366671",A


It has the following signature:

| time | value | category |
|------|-------|----------|

In [12]:
// Group it by "category"
val groupedDF = valuesDF.groupBy { category }
groupedDF

<iframe onload="o_resize_iframe_out_4()" style="width:100%;" class="result_container" id="iframe_out_4" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}


:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 <table class="dataframe" id="df_-1509949434"></table>

<p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i < df.cols.length; i++) 

Now we have a `GroupBy` with a signature

<table>
  <thead>
    <tr>
      <th>key: [category]</th>
      <th>group: DataFrame[time|value|category]</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>A</td>
      <td>A-Group</td>
    </tr>
    <tr>
      <td>B</td>
      <td>B-Group</td>
    </tr>
  </tbody>
</table>

In [13]:
groupedDF.statSmooth { x(time); y(value) }

<iframe onload="o_resize_iframe_out_5()" style="width:100%;" class="result_container" id="iframe_out_5" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}


:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 <table class="dataframe" id="df_-1509949428"></table>

<p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i < df.cols.length; i++) 

After `statSmooth` applying it's still a `GroupBy` but with different signature of `group` —
all groups have the same signature as usual `DataFrame` after `statSmooth` applying (i.e. `StatSmoothFrame`):

<table>
  <thead>
    <tr>
      <th>key: [drv]</th>
      <th>group: StaSmoothFrame</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>"A"</td>
      <td>"A"-Group</td>
    </tr>
    <tr>
      <td>"B"</td>
      <td>"B"-Group</td>
    </tr>
  </tbody>
</table>

As you can see, we did indeed do a `statSmooth` transformation within groups, the grouping keys did not change.

The plotting process doesn't change much — we do everything the same.

In [14]:
groupedDF.plot {
    statSmooth(time, value) {
        line {
            x(Stat.x)
            y(Stat.y)
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="3NGdet"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.7406060606060603,-1.6399999999999997,-1.539393939393939,-1.4387878787878785,-1.338181818181818,-1.2375757575757573,-1.1369696969696967,-1.0363636363636362,-0.9357575757575756,-0.835151515151515,-0.7345454545454544,-0.6339393939393938,-0.5333333333333332,-0.4327272727272726,-0.33212121212121204,-0.23151515151515145,-0.13090909090908998,-0.030303030303029388,0.0703030303030312,0.1709090909090918,0.2715151515151524,0.37212121212121296,0.47272727272727355,0.5733333333333341,0.6739393939393947,0.7745454545454553,0.8751515151515159,0.9757575757575765,1.076363636363637,1.1769696969696977,1.2775757575757583,1.3781818181818188,1.4787878787878794,1.57939393939394,1.6800000000000006,1.7806060606060612,1.8812121212121218,1.9818181818181824,2.082424242424243,2.1830303030303035,2.283636363636364,2.3842424242424247,2.4848484848484853,2.585454545454546,2.6860606060606065,2.786666666666667,2.8872727272727277,2.9878787878787882,3.088484848484849,3.1890909090909103,3.28969696969697,3.3903030303030315,3.490909090909091,3.5915151515151527,3.6921212121212124,3.792727272727274,3.8933333333333335,3.993939393939395,4.094545454545455,4.195151515151516,4.295757575757576,4.396363636363637,4.496969696969697,4.5975757575757585,4.69818181818182,4.79878787878788,4.899393939393941,5.000000000000001,-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.7406060606060603,-1.6399999999999997,-1.539393939393

As you can see, there are two lines because we have two groups of data.
To distinguish them, we need to add mapping to the color from the key.
This is convenient — the key is available in the context

In [15]:
groupedDF.plot {
    statSmooth(time, value, method = SmoothMethod.Polynomial(3)) {
        line {
            x(Stat.x)
            y(Stat.y)
            color(key.category)
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="MQqxyk"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y",
"color":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.7406060606060603,-1.6399999999999997,-1.539393939393939,-1.4387878787878785,-1.338181818181818,-1.2375757575757573,-1.1369696969696967,-1.0363636363636362,-0.9357575757575756,-0.835151515151515,-0.7345454545454544,-0.6339393939393938,-0.5333333333333332,-0.4327272727272726,-0.33212121212121204,-0.23151515151515145,-0.13090909090908998,-0.030303030303029388,0.0703030303030312,0.1709090909090918,0.2715151515151524,0.37212121212121296,0.47272727272727355,0.5733333333333341,0.6739393939393947,0.7745454545454553,0.8751515151515159,0.9757575757575765,1.076363636363637,1.1769696969696977,1.2775757575757583,1.3781818181818188,1.4787878787878794,1.57939393939394,1.6800000000000006,1.7806060606060612,1.8812121212121218,1.9818181818181824,2.082424242424243,2.1830303030303035,2.283636363636364,2.3842424242424247,2.4848484848484853,2.585454545454546,2.6860606060606065,2.786666666666667,2.8872727272727277,2.9878787878787882,3.088484848484849,3.1890909090909103,3.28969696969697,3.3903030303030315,3.490909090909091,3.5915151515151527,3.6921212121212124,3.792727272727274,3.8933333333333335,3.993939393939395,4.094545454545455,4.195151515151516,4.295757575757576,4.396363636363637,4.496969696969697,4.5975757575757585,4.69818181818182,4.79878787878788,4.899393939393941,5.000000000000001,-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.84121212121

The `smoothLine()` layer also works.
Moreover, if we have exactly one grouping key, a mapping from it to `color` will be created by default.

In [16]:
groupedDF.plot {
    smoothLine(time, value)
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="cmO3hv"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"time",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"value",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y",
"color":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.7406060606060603,-1.6399999999999997,-1.539393939393939,-1.4387878787878785,-1.338181818181818,-1.2375757575757573,-1.1369696969696967,-1.0363636363636362,-0.9357575757575756,-0.835151515151515,-0.7345454545454544,-0.6339393939393938,-0.5333333333333332,-0.4327272727272726,-0.33212121212121204,-0.23151515151515145,-0.13090909090908998,-0.030303030303029388,0.0703030303030312,0.1709090909090918,0.2715151515151524,0.37212121212121296,0.47272727272727355,0.5733333333333341,0.6739393939393947,0.7745454545454553,0.8751515151515159,0.9757575757575765,1.076363636363637,1.1769696969696977,1.2775757575757583,1.3781818181818188,1.4787878787878794,1.57939393939394,1.6800000000000006,1.7806060606060612,1.8812121212121218,1.9818181818181824,2.082424242424243,2.1830303030303035,2.283636363636364,2.3842424242424247,2.4848484848484853,2.585454545454546,2.6860606060606065,2.786666666666667,2.8872727272727277,2.9878787878787882,3.088484848484849,3.1890909090909103,3.28969696969697,3.3903030303030315,3.490909090909091,3.5915151515151527,3.6921212121212124,3.792727272727274,3.8933333333333335,3.993939393939395,4.094545454545455,4.195151515151516,4.295757575757576,4.396363636363637,4.496969696969697,4.5975757575757585,4.69818181818182,4.79878787878788,4.899393939393941,5.000000000000001,-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484

We can customize it like we used to. From the differences — access to `key` columns:

In [17]:
groupedDF.plot {
    smoothLine(time, value) {
        color = Color.GREEN
        type(key.category)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="NSIBuu"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"time",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"value",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"linetype"
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y",
"color":"category",
"linetype":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.7406060606060603,-1.6399999999999997,-1.539393939393939,-1.4387878787878785,-1.338181818181818,-1.2375757575757573,-1.1369696969696967,-1.0363636363636362,-0.9357575757575756,-0.835151515151515,-0.7345454545454544,-0.6339393939393938,-0.5333333333333332,-0.4327272727272726,-0.33212121212121204,-0.23151515151515145,-0.13090909090908998,-0.030303030303029388,0.0703030303030312,0.1709090909090918,0.2715151515151524,0.37212121212121296,0.47272727272727355,0.5733333333333341,0.6739393939393947,0.7745454545454553,0.8751515151515159,0.9757575757575765,1.076363636363637,1.1769696969696977,1.2775757575757583,1.3781818181818188,1.4787878787878794,1.57939393939394,1.6800000000000006,1.7806060606060612,1.8812121212121218,1.9818181818181824,2.082424242424243,2.1830303030303035,2.283636363636364,2.3842424242424247,2.4848484848484853,2.585454545454546,2.6860606060606065,2.786666666666667,2.8872727272727277,2.9878787878787882,3.088484848484849,3.1890909090909103,3.28969696969697,3.3903030303030315,3.490909090909091,3.5915151515151527,3.6921212121212124,3.792727272727274,3.8933333333333335,3.993939393939395,4.094545454545455,4.195151515151516,4.295757575757576,4.396363636363637,4.496969696969697,4.5975757575757585,4.69818181818182,4.79878787878788,4.899393939393941,5.000000000000001,-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.6

### Inside `groupBy{}` plot context

We can apply `groupBy` modification to the initial dataset and build a histogram with grouped data the same way:

In [18]:
valuesDF.plot {
    groupBy(category) {
        smoothLine(time, value)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="4mxTcH"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"time",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"value",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y",
"color":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.7406060606060603,-1.6399999999999997,-1.539393939393939,-1.4387878787878785,-1.338181818181818,-1.2375757575757573,-1.1369696969696967,-1.0363636363636362,-0.9357575757575756,-0.835151515151515,-0.7345454545454544,-0.6339393939393938,-0.5333333333333332,-0.4327272727272726,-0.33212121212121204,-0.23151515151515145,-0.13090909090908998,-0.030303030303029388,0.0703030303030312,0.1709090909090918,0.2715151515151524,0.37212121212121296,0.47272727272727355,0.5733333333333341,0.6739393939393947,0.7745454545454553,0.8751515151515159,0.9757575757575765,1.076363636363637,1.1769696969696977,1.2775757575757583,1.3781818181818188,1.4787878787878794,1.57939393939394,1.6800000000000006,1.7806060606060612,1.8812121212121218,1.9818181818181824,2.082424242424243,2.1830303030303035,2.283636363636364,2.3842424242424247,2.4848484848484853,2.585454545454546,2.6860606060606065,2.786666666666667,2.8872727272727277,2.9878787878787882,3.088484848484849,3.1890909090909103,3.28969696969697,3.3903030303030315,3.490909090909091,3.5915151515151527,3.6921212121212124,3.792727272727274,3.8933333333333335,3.993939393939395,4.094545454545455,4.195151515151516,4.295757575757576,4.396363636363637,4.496969696969697,4.5975757575757585,4.69818181818182,4.79878787878788,4.899393939393941,5.000000000000001,-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484